In [2]:
import json
import os
import logging
# These commands are for muting non-damaging errors - used for GPU
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras import layers, models, Input
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Importing all data sets with their labels

train_paths = np.load("data_splits/train_paths.npy", allow_pickle=True)
val_paths   = np.load("data_splits/val_paths.npy", allow_pickle=True)
test_paths  = np.load("data_splits/test_paths.npy", allow_pickle=True)

train_labels = np.load("data_splits/train_labels.npy", allow_pickle=True)
val_labels   = np.load("data_splits/val_labels.npy", allow_pickle=True)
test_labels  = np.load("data_splits/test_labels.npy", allow_pickle=True)

with open("data_splits/class_info.json", "r") as f:
    info = json.load(f)

class_names = info["class_names"]
num_classes = len(class_names)

In [3]:
# Creating my three datasets

AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32    # set to 32 instead of 64 for better generalization
steps_per_epoch = len(train_paths) // BATCH_SIZE

def process_path(file_path, label):
    img = tf.io.read_file(file_path)                      
    img = tf.image.decode_jpeg(img, channels=3)           
    img = tf.image.resize(img, [224,224])
    return img, label

# Training set
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.shuffle(1000).repeat().batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Validation set
val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Test set
test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

I0000 00:00:1758999864.966174    3934 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5556 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
for img, label in train_ds.take(1):
    print(img.shape)

(32, 224, 224, 3)


In [5]:
from tensorflow.keras.utils import get_file

# URL for EfficientNetB0, include_top=False
url = "https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5"
weights_path = get_file("efficientnetb0_notop.h5", url, cache_subdir="models", cache_dir="~/.keras")

In [6]:
# Importing pretrained models up until the final calssifier layer for both models
base_model_resnet = ResNet50(
    weights='imagenet',      
    include_top=False,       
    input_shape=(224, 224, 3) 
)

base_model_effb0 = EfficientNetB0(
    weights=weights_path,    # path to freshly downloaded RGB weights
    include_top=False,
    input_shape=(224, 224, 3)
)

In [7]:
# set a scheduler for smooth convergence  
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3,
    decay_steps=steps_per_epoch * 20,
    alpha=1e-5 
)

In [8]:
# My ResNet50
base_model_resnet.trainable = False   # freeze pretrained weights

model_resnet = models.Sequential([
    base_model_resnet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax') 
])

# Compile my model with optimizer, loss function and evalution metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model_resnet.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_resnet.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,851,919 (90.99 MB)

 Trainable params: 264,207 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [9]:
# My EfficientNetB0
base_model_effb0.trainable = False   # freeze pretrained weights

model_effb0 = models.Sequential([
    base_model_effb0,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax') 
])

# Compile my model with optimizer, loss function and evalution metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model_effb0.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_effb0.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,215,474 (16.08 MB)

 Trainable params: 165,903 (648.06 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [10]:
# To give each class an equal chance of being picked 
class_weights = compute_class_weight(
    "balanced", classes=np.unique(train_labels), y=train_labels
)
class_weights = dict(enumerate(class_weights))

# Early stopping in case of overfitting
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

In [11]:
# Training my ResNet50 model 
history = model_resnet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    callbacks=[early_stop],
    class_weight=class_weights
)

Epoch 1/20
  5/515 ━━━━━━━━━━━━━━━━━━━━ 17s 35ms/step - accuracy: 0.0580 - loss: 3.4383

I0000 00:00:1758999876.870894    3976 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


515/515 ━━━━━━━━━━━━━━━━━━━━ 33s 47ms/step - accuracy: 0.6652 - loss: 1.0450 - val_accuracy: 0.8794 - val_loss: 0.3695
Epoch 2/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8189 - loss: 0.5310 - val_accuracy: 0.8944 - val_loss: 0.3272
Epoch 3/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.8507 - loss: 0.4246 - val_accuracy: 0.9079 - val_loss: 0.2630
Epoch 4/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8690 - loss: 0.3824 - val_accuracy: 0.9283 - val_loss: 0.2186
Epoch 5/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.8874 - loss: 0.3161 - val_accuracy: 0.9380 - val_loss: 0.1755
Epoch 6/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.9039 - loss: 0.2763 - val_accuracy: 0.9404 - val_loss: 0.1880
Epoch 7/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9081 - loss: 0.2588 - val_accuracy: 0.9404 - val_loss: 0.1825
Epoch 8/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9162 - loss: 0.2306 - val_accurac

In [18]:
# Training my EfficientNetB0 model 
history = model_effb0.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    class_weight=class_weights
)

Epoch 1/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.8854 - loss: 0.3398 - val_accuracy: 0.9341 - val_loss: 0.2091
Epoch 2/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.9001 - loss: 0.2924 - val_accuracy: 0.9443 - val_loss: 0.1885
Epoch 3/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9103 - loss: 0.2598 - val_accuracy: 0.9482 - val_loss: 0.1724
Epoch 4/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9180 - loss: 0.2393 - val_accuracy: 0.9520 - val_loss: 0.1679
Epoch 5/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9228 - loss: 0.2207 - val_accuracy: 0.9516 - val_loss: 0.1566
Epoch 6/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9249 - loss: 0.2150 - val_accuracy: 0.9525 - val_loss: 0.1516
Epoch 7/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9312 - loss: 0.2042 - val_accuracy: 0.9540 - val_loss: 0.1454
Epoch 8/20
515/515 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9305 - loss: 0.2058 - 

In [19]:
# Final evaluation on the test set
test_loss, test_acc = model_resnet.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.9671 - loss: 0.1197
Test accuracy: 0.967


In [20]:
# Final evaluation on the test set
test_loss, test_acc = model_effb0.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9506 - loss: 0.1556
Test accuracy: 0.951


In [21]:
model_resnet.save("resnet.keras")

In [22]:
model_effb0.save("efficient.keras")